In [26]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from dotenv import load_dotenv, find_dotenv

from langchain.tools import E2BDataAnalysisTool
import os
from langchain.agents import AgentExecutor
from langchain.agents import tool, initialize_agent, AgentType, Tool
from langchain.chat_models import AzureChatOpenAI

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
E2B_API_KEY = os.getenv("E2B_API_KEY")

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_tool = TavilySearchResults(
    description="A search engine. Useful for when you don't know something and want to find it out.",)

In [5]:
e2b_data_analysis_tool = E2BDataAnalysisTool(api_key=E2B_API_KEY)
#e2b_data_analysis_tool.install_python_packages("geopy")

In [6]:
@tool
def install_python_package_to_e2b(package_name: str):
    """
    Installs a python package to the E2B environment. This is useful for when you need to install a package that is not already installed in the E2B environment.
    """
    e2b_data_analysis_tool.install_python_packages(package_name)

In [7]:
tools = [e2b_data_analysis_tool, tavily_tool, install_python_package_to_e2b]

Can a normal gpt (not assitant do the same?)

In [13]:
txt = """
'Aircraft ids: [\'KL204\', \'KL420\']\nInfo on KL204 B744 index = 0\nPos: N52\'00\'0.0"  E004\'00\'0.0"\nHdg: 270   Trk: 270\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\nInfo on KL420 B744 index = 1\nPos: N52\'00\'0.0"  E003\'12\'0.0"\nHdg: 090   Trk: 090\nAlt: 25000 ft  V/S: 0 fpm\nCAS/TAS/GS: 350/498/498 kts   M: 0.827\n\n'
"""

question = f"Based on this info what wil be the position of the closest approach point between these aircraft and after how much time will it happen: \n\n {txt}"

In [28]:
# #llm = ChatOpenAI(model='gpt-4-0125-preview', temperature=0)

# llm = AzureChatOpenAI(
#     openai_api_base='https://air-traffic-management-llm.openai.azure.com/openai',
#     openai_api_version="2023-03-15-preview",
#     deployment_name="thesis",
#     openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#     openai_api_type="azure",
#     model_name="gpt-4"
# )

# tools = [
#     Tool(
#         name="install_python_package_to_e2b",
#         func=lambda x: install_python_package_to_e2b(x),
#         description="use when you need to install a python package to the E2B environment"
#     ),

#     Tool(
#         name="a python interpreter in the E2B environment",
#         func=lambda x: e2b_data_analysis_tool.run_command(x),
#         description="useful when you need to run python code"
#     )
# ]

# react = initialize_agent(
#     tools, llm, verbose=True)

# react.run(question)

c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
c:\Users\justa\OneDrive\Desktop\Developer\LLM-Enhanced-ATM\venv_test\lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updating https://air-traffic-management-llm.openai.azure.com/openai to https://air-traffic-management-llm.openai.azure.com/openai.
  warnings.warn(




> Entering new AgentExecutor chain...
To find the position of the closest approach point between these two aircraft, I need to calculate their positions over time and see where their paths come closest to each other. Since both aircraft are flying at the same altitude and with no vertical speed (V/S), I can simplify the problem by considering only their horizontal movement.

We have two aircraft that are initially 48 minutes of longitude apart (the difference between E004'00'0.0" and E003'12'0.0"), flying towards each other on a collision course, both at a ground speed (GS) of 498 knots.

To calculate the time until the closest approach, I can use the relative speed of the two aircraft, which is the sum of their ground speeds since they are flying directly towards each other. The distance they need to cover before the closest approach is the longitudinal difference.

The longitudinal difference can be converted from minutes to nautical miles. There are 60 minutes in a degree and 1 mi

Exception: Failed to install package `geopy`: /bin/bash: line 1: geopy: command not found
ERROR: You must give at least one requirement to install (see "pip help install")

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

In [20]:
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# gpt-4-0125-preview
# Create a chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful air taffic control operator, that can also use tools to answer questions and perform tasks",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, return_intermediate_steps=True)

AttributeError: 'AzureChatOpenAI' object has no attribute 'bind_tools'

In [18]:
agent_executor.invoke(
    {"input": question})



> Entering new AgentExecutor chain...


KeyboardInterrupt: 